In [1]:
import torch
import pickle
import random
import numpy as np
import main_data_representation as mdr

In [2]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# Case 1. model = ts2vec
config1 = {
    "model": 'ts2vec',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/ts2vec.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "input_dim": 9,  # 데이터의 변수 개수, int
        "repr_dim": 64,  # data representation 차원, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        "hidden_dim": 64,  # encoder의 hidden dimension, int(default: 64, 범위: 1 이상, default 값 사용 권장)
        "num_epochs": 50,  # 학습 epoch 횟수, int(default: 50, 범위: 1 이상)
        "batch_size": 512,  # batch 크기, int(default: 512, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr": 0.001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
        "device": "cuda",  # 학습 환경, ["cuda", "cpu"] 중 선택
    }
}

# Case 2. model = ts_tcc
config2 = {
    "model": 'ts_tcc',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/ts_tcc.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "input_dim": 9,  # 데이터의 변수 개수, int
        "repr_dim": 64,  # data representation 차원, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        "hidden_dim": 100,  # temporal / contextual contrasting 모듈의 hidden dimension, int(default: 100, 범위: 1 이상, default 값 사용 권장)
        "timesteps": 6,  # temporal contrasting 모듈에서 미래 예측할 시점의 길이, int(default: 6, 범위: 1 이상)
        "num_epochs": 50,  # 학습 epoch 횟수, int(default: 50, 범위: 1 이상)
        "batch_size": 512,  # batch 크기, int(default: 512, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr": 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
        "device": "cuda",  # 학습 환경, ["cuda", "cpu"] 중 선택
        "jitter_scale_ratio": 1.1,  # time series data augementation 중 weak augementation의 강도, float(default: 1.1, default 값 사용 권장)
        "jitter_ratio": 0.8,  # time series data augementation 중 strong augementation의 강도, float(default: 0.8, default 값 사용 권장)
        "max_seg": 8  # strong augementation에서 permutation 진행시 데이터의 최대 분할 개수, int(default: 8, default 값 사용 권장)
    }
}

# Case 3. model = rae_mepc
config3 = {
    "model": 'rae_mepc',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/rae_mepc.pt',  # 학습 완료 모델 저장 경로
    "parameter": {
        "window_size": 32,  # 모델의 input sequence 길이, int(default: 32, 범위: 0 이상 & 원래 데이터의 sequence 길이 이하)
        "input_dim": 9,  # 데이터의 변수 개수, int
        "repr_dim": 64,  # data representation 차원, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        "enc_nlayers": 3,  # multi-resolution encoder를 구성하는 sub-encoder의 개수, int(default: 3, 범위: 1 이상, default 값 사용 권장)
        "dec_nlayers": 3,  # multi-resolution decoder를 구성하는 sub-decoder의 개수, int(default: 3, 범위: 1 이상, default 값 사용 권장)
        "tau": 4,  # multi-resolution encoder 및 decoder의 resolution를 조절하는 값, int(default: 4, 범위: 2 이상, default 값 사용 권장)
        "num_epochs": 50,  # 학습 epoch 횟수, int(default: 50, 범위: 1 이상)
        "batch_size": 512,  # batch 크기, int(default: 512, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr": 0.0001,  # learning rate, float(default: 0.0001, 범위: 0.1 이하)
        "device": "cuda"  # 학습 환경, ["cuda", "cpu"] 중 선택
    }
}

# Case 4. model = stoc
config4 = {
    "model": 'stoc',
    "training": True,  # 학습 여부, 저장된 학습 완료 모델 존재시 False로 설정
    "best_model_path": './ckpt/stoc.pt',  # 학습 완료 모델 저장 경로
    "parameter": { 
        "window_size": 32,  # 모델의 input sequence 길이, int(default: 32, 범위: 0 이상 & 원래 데이터의 sequence 길이 이하)
        "input_dim": 9,  # 데이터의 변수 개수, int
        "repr_dim": 64,  # data representation 차원, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
        "hidden_dim": 256,  # encoder의 hidden dimension, int(default: 256, 범위: 1 이상, default 값 사용 권장)
        "forecast_step": 1,  # 미래 시계열 데이터에 대하여 예측할 시점의 길이, int(default: 6, 범위: 1 이상)
        "num_epochs": 50,  # 학습 epoch 횟수, int(default: 50, 범위: 1 이상)
        "batch_size": 512,  # batch 크기, int(default: 512, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
        "lr": 0.001,  # learning rate, float(default: 0.001, 범위: 0.1 이하)
        "device": "cuda",  # 학습 환경, ["cuda", "cpu"] 중 선택, 
        "patience": 10,  # 예측 모델 학습 시, 사전 설정한 epoch 동안 loss가 감소하지 않으면 학습 조기 중단, int(default: 10, 범위: 1 이상 num_epochs 미만)
    }
}

In [4]:
# Dataset
dataset_dir = {
    "train": './data/X_train.pkl',
    "test": './data/X_test.pkl'
}

# train/test 데이터 불러오기 (pickle 형태)
# shape=(# observations, # features, # time steps)
train_data = pickle.load(open(dataset_dir["train"], 'rb'))  # shape=(7352, 9, 128)
test_data = pickle.load(open(dataset_dir["test"], 'rb'))  # shape=(2947, 9, 128)

In [5]:
# Case 1. model = ts2vec
config = config1
data_repr = mdr.Encode(config, train_data, test_data)
model = data_repr.build_model()  # 모델 구축

if config["training"]:
    best_model = data_repr.train_model(model)  # 모델 학습
    data_repr.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

train_repr, test_repr = data_repr.encode_data(model, best_model_path=config["best_model_path"])  # representation 도출

Start training model

Epoch #1: train loss=4.936868476867676
Epoch #1: validation loss=4.943576335906982

Epoch #2: train loss=4.627585792541504
Epoch #2: validation loss=4.884347915649414

Epoch #3: train loss=4.79036808013916
Epoch #3: validation loss=4.623116493225098

Epoch #4: train loss=4.55261926651001
Epoch #4: validation loss=4.731514930725098

Epoch #5: train loss=4.5294112205505375
Epoch #5: validation loss=4.675436973571777

Epoch #6: train loss=4.473555088043213
Epoch #6: validation loss=4.2629075050354

Epoch #7: train loss=4.341904354095459
Epoch #7: validation loss=4.463427543640137

Epoch #8: train loss=4.481424331665039
Epoch #8: validation loss=4.20048189163208

Epoch #9: train loss=4.2812458038330075
Epoch #9: validation loss=4.146800994873047

Epoch #10: train loss=4.198643016815185
Epoch #10: validation loss=4.0082621574401855

Epoch #11: train loss=4.109360694885254
Epoch #11: validation loss=4.026050567626953

Epoch #12: train loss=4.023466968536377
Epoch #12: v

In [6]:
print(train_repr.shape, test_repr.shape)

(7352, 64) (2947, 64)


In [7]:
train_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,-0.013295,0.028089,0.244010,0.198956,0.722100,1.548550,0.776289,1.434361,0.002931,-0.018699,...,0.102955,0.046750,1.009223,0.061086,0.713275,-0.044171,-0.020635,-0.001691,0.012473,-0.035363
1,0.008384,0.037686,0.244832,0.204868,0.840896,1.535955,0.815045,1.449501,0.118022,-0.032593,...,0.075448,0.024197,0.959976,-0.028372,0.784408,0.092432,0.082617,-0.013228,0.003990,-0.024973
2,0.073099,0.043213,0.215702,0.172657,0.773396,1.599531,0.699918,1.435774,0.227197,0.032223,...,0.107260,0.040140,0.971930,0.014136,0.747191,-0.018825,0.089505,0.087803,0.028894,0.056081
3,-0.048631,-0.103282,0.207749,0.199997,0.478100,1.619585,0.700773,1.454298,0.192399,0.000166,...,0.101328,-0.083185,0.926623,-0.003074,0.845482,-0.151960,0.092022,0.073950,-0.022313,-0.099365
4,0.002300,0.059146,0.229711,0.199662,0.678738,1.608757,0.743883,1.477629,0.197255,0.070348,...,0.131444,-0.074090,0.903523,0.016384,0.879731,0.035364,0.099931,0.048464,0.030324,-0.068622


In [8]:
test_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,0.604510,0.588426,-0.205228,0.671284,1.973900,1.260330,0.717597,1.013724,-0.201380,0.077659,...,0.011921,0.626707,0.719043,0.182539,0.493354,0.631235,-0.324439,0.127469,0.527457,0.404922
1,0.561814,0.511812,-0.280863,0.716274,1.867069,1.569808,0.585808,1.147880,0.075876,0.137893,...,-0.080376,0.465956,0.626413,-0.045534,0.617220,0.433061,-0.304907,0.286128,0.394571,0.266304
2,0.546373,0.062498,-0.327004,0.813788,1.416170,1.549371,0.558580,1.150737,0.253067,0.110696,...,-0.163509,0.193838,0.660893,0.044866,0.672460,0.075143,-0.237908,0.360987,0.328562,-0.031578
3,0.512255,0.054458,-0.322989,0.861922,1.365365,1.539171,0.485873,1.161082,0.469912,0.074668,...,-0.154426,0.279764,0.655488,0.167106,0.683768,0.027333,-0.257095,0.466250,0.340417,0.070660
4,0.483263,-0.091438,-0.338230,0.893001,1.063844,1.575519,0.512306,1.160689,0.390473,0.104820,...,-0.136991,0.159172,0.606764,0.117792,0.785372,-0.158917,-0.198069,0.467135,0.286246,-0.143399


In [10]:
# Case 2. model = ts_tcc
config = config2
data_repr = mdr.Encode(config, train_data, test_data)
model = data_repr.build_model()  # 모델 구축

if config["training"]:
    best_model = data_repr.train_model(model)  # 모델 학습
    data_repr.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

train_repr, test_repr = data_repr.encode_data(model, best_model_path=config["best_model_path"])  # representation 도출

Start training model

Epoch #1: train loss=19.582977294921875
Epoch #1: validation loss=19.5350341796875

Epoch #2: train loss=19.56104278564453
Epoch #2: validation loss=19.354076385498047

Epoch #3: train loss=19.525501251220703
Epoch #3: validation loss=20.442901611328125

Epoch #4: train loss=19.49390983581543
Epoch #4: validation loss=19.357648849487305

Epoch #5: train loss=19.48751449584961
Epoch #5: validation loss=19.4041690826416

Epoch #6: train loss=19.45798110961914
Epoch #6: validation loss=19.630054473876953

Epoch #7: train loss=19.44181251525879
Epoch #7: validation loss=19.73177719116211

Epoch #8: train loss=19.39997100830078
Epoch #8: validation loss=20.054264068603516

Epoch #9: train loss=19.373287200927734
Epoch #9: validation loss=19.525205612182617

Epoch #10: train loss=19.33846664428711
Epoch #10: validation loss=19.444679260253906

Epoch #11: train loss=19.32611656188965
Epoch #11: validation loss=19.415756225585938

Epoch #12: train loss=19.261075973510742


In [11]:
print(train_repr.shape, test_repr.shape)

(7352, 64) (2947, 64)


In [12]:
train_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,0.744626,0.0,0.0,0.829155,1.488290,0.973557,0.0,1.981022,1.103711,1.479490,...,0.0,1.870526,1.530592,1.190534,0.524825,0.0,0.192903,0.654648,0.0,1.533573
1,0.726559,0.0,0.0,0.832423,1.431869,1.024122,0.0,1.989928,1.070720,1.485404,...,0.0,1.854669,1.538515,1.211092,0.509399,0.0,0.258388,0.676524,0.0,1.532583
2,0.721764,0.0,0.0,0.819706,1.413890,1.023439,0.0,1.995048,1.068986,1.473787,...,0.0,1.847066,1.532187,1.199001,0.509043,0.0,0.270424,0.676852,0.0,1.532341
3,0.705584,0.0,0.0,0.821733,1.390979,1.034416,0.0,2.003811,1.071950,1.477579,...,0.0,1.859940,1.531603,1.199053,0.515407,0.0,0.286682,0.677910,0.0,1.527292
4,0.713996,0.0,0.0,0.828449,1.396067,1.035022,0.0,2.001043,1.058323,1.483781,...,0.0,1.865437,1.531960,1.196236,0.518705,0.0,0.289590,0.683015,0.0,1.521953


In [13]:
test_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,1.019318,0.0,0.0,0.835575,1.651087,0.785979,0.017311,2.017650,1.134924,1.429579,...,0.0,1.809016,1.544034,1.173692,0.443830,0.030953,0.280080,0.788252,0.0,1.467388
1,0.729495,0.0,0.0,0.828378,1.448049,1.071115,0.000000,1.984238,1.010257,1.432151,...,0.0,1.840703,1.527592,1.154068,0.457090,0.000000,0.305323,0.679145,0.0,1.599990
2,0.736129,0.0,0.0,0.823715,1.361779,1.069044,0.000000,2.005557,1.029939,1.418222,...,0.0,1.824186,1.515320,1.159892,0.465052,0.000000,0.364472,0.692511,0.0,1.554687
3,0.749357,0.0,0.0,0.817371,1.392371,1.060552,0.000000,1.990492,1.039663,1.420777,...,0.0,1.827514,1.516359,1.148880,0.457974,0.000000,0.328887,0.695106,0.0,1.554501
4,0.735662,0.0,0.0,0.831737,1.356451,1.077132,0.000000,1.999531,1.031571,1.438225,...,0.0,1.850438,1.516683,1.140328,0.449178,0.000000,0.349859,0.700667,0.0,1.555602


In [15]:
# Case 3. model = rae_mepc
config = config3
data_repr = mdr.Encode(config, train_data, test_data)
model = data_repr.build_model()  # 모델 구축

if config["training"]:
    best_model = data_repr.train_model(model)  # 모델 학습
    data_repr.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

train_repr, test_repr = data_repr.encode_data(model, best_model_path=config["best_model_path"])  # representation 도출

Start training model

Epoch #1: train loss=0.325350284576416
Epoch #1: validation loss=0.1690431386232376

Epoch #2: train loss=0.3037114143371582
Epoch #2: validation loss=0.15433645248413086

Epoch #3: train loss=0.28291401267051697
Epoch #3: validation loss=0.1394573450088501

Epoch #4: train loss=0.26264896988868713
Epoch #4: validation loss=0.1264355331659317

Epoch #5: train loss=0.2424534112215042
Epoch #5: validation loss=0.11284668743610382

Epoch #6: train loss=0.22465625405311584
Epoch #6: validation loss=0.10531354695558548

Epoch #7: train loss=0.2125057578086853
Epoch #7: validation loss=0.10442522168159485

Epoch #8: train loss=0.2028370499610901
Epoch #8: validation loss=0.1033286601305008

Epoch #9: train loss=0.193651482462883
Epoch #9: validation loss=0.10279160737991333

Epoch #10: train loss=0.18893495202064514
Epoch #10: validation loss=0.10294356942176819

Epoch #11: train loss=0.18549083173274994
Epoch #11: validation loss=0.10147477686405182

Epoch #12: train l

In [16]:
print(train_repr.shape, test_repr.shape)

(7352, 64) (2947, 64)


In [17]:
train_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,-0.688607,0.061593,-0.383556,0.948981,-0.580462,0.291508,-0.491042,0.131423,-0.563544,-0.440816,...,-1.158566,0.718064,-0.633171,-1.037993,-0.654820,-1.400875,1.300124,0.142990,1.256600,0.338931
1,-0.703893,0.073500,-0.387369,0.938212,-0.573998,0.286079,-0.479039,0.131423,-0.562809,-0.424822,...,-1.158379,0.716326,-0.631527,-1.052797,-0.672248,-1.400875,1.300124,0.121656,1.239977,0.322619
2,-0.712129,0.083974,-0.391321,0.930104,-0.573998,0.286425,-0.471686,0.127747,-0.556572,-0.417296,...,-1.158379,0.719549,-0.631527,-1.064433,-0.679306,-1.402853,1.308658,0.114472,1.234172,0.313113
3,-0.722868,0.083974,-0.386363,0.943948,-0.589505,0.293000,-0.461986,0.132758,-0.555541,-0.417296,...,-1.161435,0.719549,-0.635632,-1.060184,-0.664306,-1.408921,1.317508,0.101543,1.224174,0.308647
4,-0.704933,0.074808,-0.380772,0.956162,-0.586930,0.299106,-0.461986,0.138202,-0.555541,-0.426082,...,-1.168403,0.718252,-0.636969,-1.049286,-0.656282,-1.403172,1.321968,0.119792,1.240214,0.322928


In [18]:
test_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,-0.557617,0.160285,-0.425887,0.864598,-0.602728,0.237714,-0.521347,0.143738,-0.537697,-0.372173,...,-1.094319,0.778685,-0.662305,-1.071405,-0.717448,-1.414683,1.224193,0.227640,1.294488,0.406175
1,-0.669349,0.160285,-0.419329,0.874844,-0.614556,0.232385,-0.493306,0.100120,-0.511537,-0.372173,...,-1.094319,0.793442,-0.654049,-1.082455,-0.717448,-1.427174,1.265099,0.152401,1.204864,0.337063
2,-0.698920,0.143665,-0.419329,0.881700,-0.648730,0.232652,-0.477078,0.095774,-0.511537,-0.393631,...,-1.102755,0.793442,-0.653244,-1.094098,-0.710921,-1.452243,1.265099,0.125598,1.189459,0.300597
3,-0.698920,0.137622,-0.405124,0.938751,-0.648730,0.243397,-0.446895,0.101446,-0.499092,-0.394289,...,-1.095365,0.791849,-0.653244,-1.071329,-0.667992,-1.452243,1.302684,0.125598,1.189459,0.306359
4,-0.755224,0.133954,-0.398618,0.967627,-0.650972,0.255839,-0.445786,0.117957,-0.499092,-0.394289,...,-1.095365,0.783666,-0.657758,-1.049908,-0.653448,-1.457068,1.308867,0.112376,1.161752,0.337753


In [20]:
# Case 4. model = stoc
config = config4
data_repr = mdr.Encode(config, train_data, test_data)
model = data_repr.build_model()  # 모델 구축

if config["training"]:
    best_model = data_repr.train_model(model)  # 모델 학습
    data_repr.save_model(best_model, best_model_path=config["best_model_path"])  # 모델 저장

train_repr, test_repr = data_repr.encode_data(model, best_model_path=config["best_model_path"])  # representation 도출

Start training model

Epoch #1: loss=1.6170785913572592
Epoch #1: validation loss=0.10995912365615368

Epoch #2: loss=0.10599795334479388
Epoch #2: validation loss=0.09988718666136265

Epoch #3: loss=0.09985399553004433
Epoch #3: validation loss=0.09597376547753811

Epoch #4: loss=0.09701299755012288
Epoch #4: validation loss=0.09307925403118134

Epoch #5: loss=0.09459449394660838
Epoch #5: validation loss=0.09128953516483307

Epoch #6: loss=0.09341227570000817
Epoch #6: validation loss=0.09153063222765923

Epoch #7: loss=0.09308350787443273
Epoch #7: validation loss=0.09027368389070034

Epoch #8: loss=0.09114879808005165
Epoch #8: validation loss=0.08633670583367348

Epoch #9: loss=0.08312011115691241
Epoch #9: validation loss=0.0729949064552784

Epoch #10: loss=0.07183623401557698
Epoch #10: validation loss=0.0637363288551569

Epoch #11: loss=0.05888085001531769
Epoch #11: validation loss=0.04443054273724556

Epoch #12: loss=0.04168248264228597
Epoch #12: validation loss=0.0309692542

In [21]:
print(train_repr.shape, test_repr.shape)

(7352, 64) (2947, 64)


In [22]:
train_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,-0.891249,-0.621638,0.417513,-0.284545,-0.251119,-0.463985,0.285838,-0.246278,-0.199712,-0.933221,...,0.078440,0.819481,-0.111247,0.792588,-0.989658,0.739370,0.144455,0.338889,0.270035,-0.108632
1,-0.895377,-0.621640,0.430735,-0.274166,-0.251016,-0.459078,0.283579,-0.248442,-0.206698,-0.933221,...,0.090645,0.827522,-0.116565,0.808396,-0.994202,0.749294,0.121209,0.340605,0.256826,-0.114984
2,-0.903910,-0.621640,0.446909,-0.274166,-0.249243,-0.459078,0.286711,-0.250495,-0.206698,-0.933654,...,0.090645,0.828098,-0.121316,0.813629,-0.992169,0.753885,0.113314,0.341678,0.253055,-0.108251
3,-0.910284,-0.625641,0.446909,-0.280113,-0.247262,-0.460682,0.288947,-0.244630,-0.217923,-0.942812,...,0.070928,0.828098,-0.119105,0.813629,-0.986798,0.757730,0.103864,0.342186,0.253055,-0.097807
4,-0.899026,-0.625162,0.441594,-0.281975,-0.247262,-0.464881,0.293224,-0.244630,-0.217923,-0.942812,...,0.099923,0.824098,-0.119105,0.812651,-0.985407,0.757744,0.109550,0.343002,0.253918,-0.093566


In [23]:
test_repr.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64
0,-0.828890,-0.454174,0.455766,-0.247606,-0.188076,-0.224023,0.312756,-0.212926,-0.009786,-0.900262,...,0.255959,0.828395,-0.084750,0.765140,-0.795153,0.716207,0.538449,0.301600,0.232086,0.212344
1,-0.941897,-0.660175,0.490743,-0.265766,-0.188076,-0.415722,0.312756,-0.212926,-0.102622,-0.900262,...,0.187805,0.831313,-0.099164,0.799842,-1.000088,0.732326,0.264744,0.301735,0.213469,-0.025478
2,-0.958226,-0.660062,0.502027,-0.287051,-0.187320,-0.410712,0.294332,-0.250739,-0.169461,-0.938403,...,0.083367,0.831313,-0.107606,0.811020,-0.972163,0.741404,0.140587,0.305364,0.211290,-0.041757
3,-0.958226,-0.660062,0.510766,-0.290792,-0.180015,-0.403985,0.298800,-0.250739,-0.169461,-0.938403,...,0.083367,0.838909,-0.107058,0.821125,-0.972163,0.747084,0.140587,0.312955,0.199819,-0.041757
4,-0.943833,-0.665081,0.510912,-0.290792,-0.179064,-0.403985,0.305384,-0.253059,-0.203349,-0.965674,...,0.086085,0.838909,-0.105862,0.822775,-0.971566,0.747084,0.135664,0.317206,0.199819,-0.032188
